In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
from io import StringIO
import re
import os

# Initialize an empty list to collect DataFrames
all_dataframes = []

# Loop through years 2015 to 2025
for year in range(2015, 2026):
    filename = f"t05st09_2_e_{year}.html"
    if os.path.exists(filename):
        try:
            with open(filename, "r", encoding="big5", errors="ignore") as file:
                soup = BeautifulSoup(file, "html.parser")

            html_io = StringIO(str(soup))
            tables = pd.read_html(html_io)

            if len(tables) > 2:
                df = tables[2]
                df.dropna(how='all', inplace=True)
                df = df[df[df.columns[0]] != df.columns[0]]
                df.iloc[:, 0] = df.iloc[:, 0].fillna("Company").replace("", "Company")
                df.columns = df.iloc[1]  # Set second row as header
                df = df[2:]              # Remove the first two rows
                df.reset_index(drop=True, inplace=True)
                df.insert(0, "Year", str(year))
                all_dataframes.append(df)
        except Exception as e:
            print(f"Failed to process {filename}: {e}")

# Concatenate all DataFrames
if all_dataframes:
    combined_df = pd.concat(all_dataframes, ignore_index=True)
    combined_df.to_csv("taiwan_stock_dividends_2015_2025.csv", index=False)
    print("Combined CSV file 'all_years_taiwan_stock_dividends.csv' has been created successfully.")
else:
    print("No valid files found to process.")


In [ ]:
import plotly.express as px

stock_list = ['1233','1303','1473','1477','2103']
df_pic = combined_df[combined_df['Company code'].isin(stock_list)]
df_pic_rename = df_pic.rename(columns={'Cash dividends distributed from retained earnings (NTD/share)' : 'Cash div'})
df_pic_year_cash_div = df_pic_rename[['Company code','Year','Cash div']]
df_pic_year_cash_div.loc[:,'Cash div'] = df_pic_year_cash_div['Cash div'].astype(float)
df_pic_year_cash_div.to_csv('dividend_year_1233_1303_1473_2103.csv')
# Create a line chart of Cash Dividend over Years for each Company Code
fig = px.line(df_pic_year_cash_div, x='Year', y='Cash div', color='Company code',
              title='Cash Dividend Over Years by Company Code',
              markers=True)
fig.show()

In [ ]:
df_pic = combined_df.sort_values(by='Year')
df_pic_rename = df_pic.rename(columns={'Cash dividends distributed from retained earnings (NTD/share)' : 'Cash div'})
df_filtered = df_pic_rename[['Company code','Year','Cash div']]
df_filtered.loc[:,'Cash div'] = df_filtered['Cash div'].astype(float)

# Calculate average cash dividend for each stock
avg_divs = df_filtered.groupby('Company code')['Cash div'].mean()

# Filter stocks with average cash dividend > 0.5
valid_stocks = avg_divs[avg_divs > 0.5].index

# Filter the original data to include only valid stocks
df_valid = df_filtered[df_filtered['Company code'].isin(valid_stocks)]

# Calculate and sort standard deviation of cash dividends for valid stocks
std_devs = df_valid.groupby('Company code')['Cash div'].std().sort_values()

std_devs_filtered = std_devs.dropna()
std_devs_filtered = std_devs_filtered[std_devs_filtered != 0]
print("Standard deviations of yearly cash dividends (average > 0.5, sorted from low to high):")
print(std_devs_filtered)